# Práctica de Laboratorio: Esteganografía LSB

**Estudiante:** Rodrigo Perez Campesino
**Fecha:** 8 de Febrero de 2026  

---

Este laboratorio implementa esteganografía LSB (Least Significant Bit) con las siguientes mejoras de seguridad:

- **Cifrado AES-256-GCM**: Todo mensaje se cifra antes de ocultarse
- **Posiciones aleatorias**: No se usa inserción secuencial, sino posiciones determinísticas pseudo-aleatorias
- **KDF (PBKDF2)**: Derivación segura de claves desde contraseña

Esto hace el sistema mucho más robusto que LSB básico.

## PARTE 1: Generador de Dataset

Este código genera 100 imágenes con ruido aleatorio y oculta un mensaje cifrado en UNA de ellas de forma aleatoria.

In [4]:
from PIL import Image
import numpy as np
import os
import random
import hashlib
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes


class SimpleStego:
    """Sistema simplificado de esteganografía LSB con AES"""
    
    def __init__(self, password):
        # Derivar clave AES desde password
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'lab_salt_2026',
            iterations=100000
        )
        self.aes_key = kdf.derive(password.encode('utf-8'))
        
        # Derivar seed para posiciones aleatorias
        seed_bytes = hashlib.sha256(self.aes_key).digest()[:4]
        self.seed = int.from_bytes(seed_bytes, 'big')
    
    def encrypt_message(self, message):
        """Cifra mensaje con AES-GCM"""
        aesgcm = AESGCM(self.aes_key)
        nonce = os.urandom(12)
        ciphertext = aesgcm.encrypt(nonce, message.encode('utf-8'), None)
        return nonce + ciphertext
    
    def get_random_positions(self, total_capacity, num_bits):
        """Genera posiciones aleatorias determinísticas"""
        rng = np.random.RandomState(self.seed)
        all_positions = np.arange(total_capacity)
        rng.shuffle(all_positions)
        return all_positions[:num_bits]


def create_noise_image(filename, width=1280, height=720):
    """Crea imagen con píxeles RGB aleatorios"""
    pixels = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)
    img = Image.fromarray(pixels, 'RGB')
    img.save(filename, 'PNG')


def hide_lsb(image_path, message, password):
    """Oculta mensaje en imagen usando LSB con cifrado y posiciones aleatorias"""
    stego = SimpleStego(password)
    
    # Cifrar mensaje
    encrypted = stego.encrypt_message(message)
    
    # Convertir a binario (con longitud al inicio)
    length_bits = format(len(encrypted), '032b')
    data_bits = ''.join(format(byte, '08b') for byte in encrypted)
    delimiter = '1' * 16
    message_binary = length_bits + data_bits + delimiter
    
    # Abrir imagen
    img = Image.open(image_path)
    pixels = np.array(img)
    height, width = pixels.shape[:2]
    
    # Calcular capacidad
    total_capacity = width * height * 3
    
    if len(message_binary) > total_capacity:
        raise ValueError("Mensaje demasiado grande")
    
    # Generar posiciones aleatorias
    positions = stego.get_random_positions(total_capacity, len(message_binary))
    
    # Insertar bits en LSB
    for i, pos in enumerate(positions):
        pixel_idx = pos // 3
        channel = pos % 3
        y = pixel_idx // width
        x = pixel_idx % width
        
        bit = int(message_binary[i])
        pixels[y, x, channel] = (pixels[y, x, channel] & 0xFE) | bit
    
    # Guardar imagen
    stego_img = Image.fromarray(pixels, 'RGB')
    stego_img.save(image_path, 'PNG')


def generate_dataset(folder, num_images, secret_message, password):
    """Genera dataset completo"""
    os.makedirs(folder, exist_ok=True)
    
    # Generar imágenes con ruido
    print(f"Generando {num_images} imágenes...")
    for i in range(num_images):
        filename = os.path.join(folder, f"image_{i:03d}.png")
        create_noise_image(filename)
    
    # Seleccionar una aleatoriamente
    secret_index = random.randint(0, num_images - 1)
    secret_image = os.path.join(folder, f"image_{secret_index:03d}.png")
    
    # Ocultar mensaje
    print("Ocultando mensaje cifrado...")
    hide_lsb(secret_image, secret_message, password)
    
    print(f"✓ Dataset creado: {num_images} imágenes en '{folder}/'")
    print("✓ Una contiene el mensaje cifrado")


# CONFIGURACIÓN
FOLDER = "dataset_images"
NUM_IMAGES = 100
FLAG = "FLAG{LSB_Stego_2026}"
PASSWORD = "password_lab_2026"

# GENERAR DATASET
generate_dataset(FOLDER, NUM_IMAGES, FLAG, PASSWORD)

Generando 100 imágenes...
Ocultando mensaje cifrado...
✓ Dataset creado: 100 imágenes en 'dataset_images/'
✓ Una contiene el mensaje cifrado


### Explicación del Generador

**¿Cómo funciona?**

1. **Derivación de Claves (KDF)**:
   - Usa PBKDF2 con 100,000 iteraciones para derivar una clave AES-256 desde la contraseña
   - De esta clave deriva un seed determinístico para el generador de posiciones aleatorias

2. **Cifrado del Mensaje**:
   - El mensaje se cifra con AES-GCM antes de ocultarse
   - Esto proporciona confidencialidad e integridad

3. **Posiciones Aleatorias**:
   - No se insertan bits secuencialmente en la imagen
   - Se genera una permutación completa de todas las posiciones disponibles
   - Se usan las primeras N posiciones de esta permutación
   - Esto hace MUCHO más difícil detectar el patrón

4. **Estructura del Payload**:
   ```
   [Longitud: 32 bits] [Datos cifrados: N bytes] [Delimitador: 16 bits]
   ```

**Ventajas sobre LSB básico**:
- Confidencialidad: Mensaje cifrado con AES-256
- Integridad: GCM detecta modificaciones
- Resistencia a análisis: Posiciones aleatorias dificultan detección estadística
- Autenticación: Solo quien conoce la contraseña puede descifrar

## PARTE 2: Detector Forense

Este código implementa dos estrategias de búsqueda del mensaje oculto.

In [5]:
import time

# Reutilizamos la clase SimpleStego definida arriba

def extract_lsb(image_path, num_bits=None, password=None):
    """Extrae bits LSB usando posiciones aleatorias"""
    img = Image.open(image_path)
    pixels = np.array(img)
    height, width = pixels.shape[:2]
    
    total_capacity = width * height * 3
    
    if num_bits is None:
        num_bits = total_capacity
    
    # Obtener posiciones
    stego = SimpleStego(password)
    positions = stego.get_random_positions(total_capacity, num_bits)
    
    # Extraer bits
    extracted_bits = []
    for pos in positions:
        pixel_idx = pos // 3
        channel = pos % 3
        y = pixel_idx // width
        x = pixel_idx % width
        
        bit = pixels[y, x, channel] & 1
        extracted_bits.append(str(bit))
    
    return ''.join(extracted_bits)


def binary_to_bytes(bits):
    """Convierte string de bits a bytes"""
    byte_list = []
    for i in range(0, len(bits), 8):
        byte_bits = bits[i:i+8]
        if len(byte_bits) == 8:
            byte_list.append(int(byte_bits, 2))
    return bytes(byte_list)


def search_brute_force(folder, password, header="FLAG"):
    """Búsqueda por fuerza bruta - analiza TODAS las imágenes completamente"""
    start_time = time.time()
    
    images = sorted([f for f in os.listdir(folder) if f.endswith('.png')])
    stego = SimpleStego(password)
    
    print(f"Búsqueda fuerza bruta: {len(images)} imágenes")
    
    for i, img_name in enumerate(images):
        img_path = os.path.join(folder, img_name)
        
        if (i + 1) % 20 == 0:
            print(f"  Progreso: {i+1}/{len(images)}")
        
        try:
            # Extraer TODOS los bits
            bits = extract_lsb(img_path, password=password)
            
            # Leer longitud
            if len(bits) < 32:
                continue
            
            data_length = int(bits[:32], 2)
            
            if data_length <= 0 or data_length > 10000:
                continue
            
            # Extraer datos
            data_bits_needed = 32 + (data_length * 8)
            if len(bits) < data_bits_needed:
                continue
            
            data_bits = bits[32:data_bits_needed]
            encrypted_data = binary_to_bytes(data_bits)
            
            # Descifrar
            message = stego.decrypt_message(encrypted_data)
            
            if message and header in message:
                elapsed = time.time() - start_time
                print(f"\n✓ Encontrado en {img_name}")
                return img_name, message, elapsed
                
        except:
            continue
    
    elapsed = time.time() - start_time
    return None, None, elapsed


def search_optimized(folder, password, header="FLAG"):
    """Búsqueda optimizada - solo extrae bytes necesarios"""
    start_time = time.time()
    
    images = sorted([f for f in os.listdir(folder) if f.endswith('.png')])
    stego = SimpleStego(password)
    
    # Solo extraer bits suficientes para verificación inicial
    header_check_bits = 32 + (50 * 8)
    
    print(f"Búsqueda optimizada: {len(images)} imágenes")
    print(f"Extracción inicial: solo {header_check_bits} bits")
    
    for i, img_name in enumerate(images):
        img_path = os.path.join(folder, img_name)
        
        if (i + 1) % 20 == 0:
            print(f"  Progreso: {i+1}/{len(images)}")
        
        try:
            # Paso 1: Extraer solo bits iniciales (RÁPIDO)
            bits_quick = extract_lsb(img_path, num_bits=header_check_bits, password=password)
            
            if len(bits_quick) < 32:
                continue
            
            data_length = int(bits_quick[:32], 2)
            
            if data_length <= 0 or data_length > 10000:
                continue
            
            # Paso 2: Si longitud válida, extraer mensaje completo
            full_bits_needed = 32 + (data_length * 8) + 16
            bits_full = extract_lsb(img_path, num_bits=full_bits_needed, password=password)
            
            data_bits = bits_full[32:32 + (data_length * 8)]
            encrypted_data = binary_to_bytes(data_bits)
            
            # Paso 3: Descifrar y verificar
            message = stego.decrypt_message(encrypted_data)
            
            if message and header in message:
                elapsed = time.time() - start_time
                print(f"\n✓ Encontrado en {img_name}")
                return img_name, message, elapsed
                
        except:
            continue
    
    elapsed = time.time() - start_time
    return None, None, elapsed


# Método para descifrado (necesario para SimpleStego)
def decrypt_message_method(self, encrypted_data):
    try:
        nonce = encrypted_data[:12]
        ciphertext = encrypted_data[12:]
        aesgcm = AESGCM(self.aes_key)
        plaintext = aesgcm.decrypt(nonce, ciphertext, None)
        return plaintext.decode('utf-8')
    except:
        return None

# Añadir método a la clase
SimpleStego.decrypt_message = decrypt_message_method

### Explicación del Detector

**Dos estrategias implementadas:**

**1. Fuerza Bruta (Naive)**:
- Analiza CADA imagen completamente
- Extrae TODOS los bits LSB de cada imagen
- Intenta descifrar y buscar el header "FLAG{LSB_Stego_2026}"
- Complejidad: O(N × W × H) donde N = número de imágenes

**2. Optimizada (Early Termination)**:
- Primero extrae solo los bits necesarios para verificar la longitud
- Si la longitud parece inválida, pasa a la siguiente imagen
- Solo si la longitud es válida extrae el mensaje completo
- Complejidad: O(N × k) donde k << W × H

**¿Por qué la optimizada es más rápida?**

En la mayoría de imágenes (99 de 100), el campo de longitud contendrá basura. Detectamos esto rápido y saltamos a la siguiente imagen sin procesar todos los píxeles.

**Consideraciones con posiciones aleatorias:**

Aunque usamos posiciones aleatorias, seguimos necesitando la contraseña correcta para:
1. Generar las mismas posiciones que usó el encoder
2. Descifrar el mensaje con AES-GCM

Sin la contraseña correcta, es computacionalmente inviable encontrar el mensaje.

## PARTE 3: Experimentos y Medición de Tiempos

Ahora ejecutamos ambos métodos y medimos tiempos.

In [6]:
# EJECUTAR BÚSQUEDAS

PASSWORD = "password_lab_2026"  # Debe coincidir con el generador
FOLDER = "dataset_images"

print("="*60)
print("MÉTODO 1: FUERZA BRUTA")
print("="*60)

img1, msg1, time1 = search_brute_force(FOLDER, PASSWORD)

if msg1:
    print(f"\nMensaje: {msg1}")
    print(f"Tiempo: {time1:.4f} segundos")
else:
    print("\nNo encontrado")
    print(f"Tiempo: {time1:.4f} segundos")

print("\n" + "="*60)
print("MÉTODO 2: OPTIMIZADO")
print("="*60)

img2, msg2, time2 = search_optimized(FOLDER, PASSWORD)

if msg2:
    print(f"\nMensaje: {msg2}")
    print(f"Tiempo: {time2:.4f} segundos")
else:
    print("\nNo encontrado")
    print(f"Tiempo: {time2:.4f} segundos")

print("\n" + "="*60)
print("COMPARACIÓN")
print("="*60)

if time1 > 0 and time2 > 0:
    speedup = time1 / time2
    print(f"Fuerza Bruta: {time1:.4f} seg")
    print(f"Optimizado:   {time2:.4f} seg")
    print(f"Mejora:       {speedup:.2f}x más rápido")

MÉTODO 1: FUERZA BRUTA
Búsqueda fuerza bruta: 100 imágenes
  Progreso: 20/100
  Progreso: 40/100
  Progreso: 60/100

✓ Encontrado en image_065.png

Mensaje: FLAG{LSB_Stego_2026}
Tiempo: 244.7284 segundos

MÉTODO 2: OPTIMIZADO
Búsqueda optimizada: 100 imágenes
Extracción inicial: solo 432 bits
  Progreso: 20/100
  Progreso: 40/100
  Progreso: 60/100

✓ Encontrado en image_065.png

Mensaje: FLAG{LSB_Stego_2026}
Tiempo: 25.0655 segundos

COMPARACIÓN
Fuerza Bruta: 244.7284 seg
Optimizado:   25.0655 seg
Mejora:       9.76x más rápido


## Resultados de Tiempos

| W x H   | Fuerza Bruta (sec) | Optimizado (sec) |
|---------|--------------------|------------------|
| 200x200 | 33.5705            | 1.6054           |
| 500x400 | 8.4581             | 1.3704           |
| 1280x720| 244.7284           | 25.0655          |


# Respuestas a las Preguntas de Análisis

### Pregunta 1: Complejidad Teórica

**(a) ¿Cuál es la complejidad  del enfoque de fuerza bruta?**

La complejidad del enfoque de fuerza bruta es $T_{\text{brute}}(N, W, H) = \mathcal{O}(N \times W \times H)$.

* **Justificación:** En el enfoque "Naive" o ingenuo, el algoritmo debe procesar cada una de las $N$ imágenes. Para cada imagen, debe extraer todos los bits LSB de cada píxel antes de reconstruir el mensaje y verificar si contiene la bandera. Dado que el número total de píxeles es el producto del ancho ($W$) por el alto ($H$), y esto se repite $N$ veces, el tiempo de ejecución crece linealmente con el volumen total de píxeles del conjunto de datos.



**(b) ¿Cuál es la complejidad  del enfoque optimizado?**

La complejidad del enfoque optimizado es $T_{\text{opt}}(N) = \mathcal{O}(N)$ en el caso promedio.

* **Justificación:** El algoritmo optimizado implementa "Terminación Temprana". Para cada una de las $N$ imágenes, solo extrae los primeros $k$ bytes (el tamaño del encabezado).
* Si el encabezado no coincide (lo cual ocurre en la inmensa mayoría de los casos), el algoritmo se detiene inmediatamente y pasa a la siguiente imagen.
* Solo en el peor de los casos (cuando se encuentra la imagen correcta), la complejidad sería $\mathcal{O}(W \times H)$, pero dado que $k \ll W \times H$, el término dominante para el conjunto de datos completo es $\mathcal{O}(N)$.





**(c) Si $W \times H$ y $k$, ¿cuántas veces es más rápido el enfoque optimizado?**

El enfoque optimizado es teóricamente **200,000 veces más rápido** por imagen.

* **Cálculo:**
* Operaciones Fuerza Bruta (por imagen): Procesar $W \times H$ píxeles.
* Operaciones Optimizado (por imagen): Procesar $k$ bytes (asumiendo 1 byte por "unidad" de verificación de cabecera para simplificar, o sus bits correspondientes).
* Factor de Speedup:

\[
\text{Speedup} = \frac{W \times H}{k} = 200{,}000
\]

* Este cálculo asume que el tiempo de procesamiento es puramente proporcional a los datos leídos y desprecia el tiempo constante de apertura de archivo.





---

### Pregunta 2: Cuellos de Botella

**(a) ¿Qué operación toma más tiempo: leer el archivo () o calcular los bits (CPU)?**

Generalmente, **leer el archivo (I/O)** es el cuello de botella dominante, especialmente para imágenes pequeñas o medianas.

* **Justificación:** El acceso al disco (incluso SSD) es órdenes de magnitud más lento que las operaciones de CPU (operaciones bitwise como AND/OR/SHIFT usadas en LSB). La CPU pasa la mayor parte del tiempo "esperando" a que los datos se carguen desde el almacenamiento a la memoria RAM.



**(b) ¿Cómo podrías verificar tu respuesta empíricamente?**

Se puede verificar mediante un experimento de **profiling** (perfilado) de código:

1. Medir el tiempo $T_{\text{total}}$ de la función completa.
2. Medir el tiempo solo de la instrucción `Image.open(path)` y `image.load()` ($T_{\text{I/O}}$).
3. Comparar: Si $T_{\text{I/O}} \approx T_{\text{total}}$, entonces el cuello de botella es I/O.
4. Alternativamente, se puede ejecutar el script monitoreando el uso del sistema: si la CPU no está al 100% pero el disco tiene alta actividad, es un cuello de botella de I/O.



**(c) Si el cuello de botella es , ¿ayudaría un procesador más rápido?**

**No significativamente.**

* **Justificación:** Según la Ley de Amdahl, si el componente que limita la velocidad es la transferencia de datos (I/O), aumentar la velocidad de procesamiento (CPU) solo mejorará la pequeña fracción de tiempo dedicada al cálculo. El sistema seguirá esperando a que el disco entregue los datos. La solución sería usar un almacenamiento más rápido (NVMe SSD) o cargar el dataset en RAM (RAM Disk).



---

### Pregunta 3: Escalabilidad

**(a) Si tuvieras 1 millón de imágenes, ¿cuánto tardaría tu algoritmo optimizado?**

*Nota: Esta respuesta depende de tus mediciones en la Parte 3, pero aquí se presenta la fórmula de proyección requerida.*

El tiempo estimado $T_{1M}$ se calcula mediante extrapolación lineal:

$T_{1M} = T_{100} \times \frac{1{,}000{,}000}{100}$

Donde $T_{100}$ es el tiempo medido para 100 imágenes en el modo optimizado. Debido a la complejidad lineal $\mathcal{O}(N)$, el tiempo crece proporcionalmente al número de imágenes.

**(b) ¿Cómo podrías usar multiprocesamiento para acelerar la búsqueda?**

Se puede utilizar la librería `multiprocessing` de Python (específicamente `Pool`) para paralelizar la carga de trabajo.

* **Estrategia:** Dividir la lista de 1 millón de rutas de archivo en $P$ sub-listas (donde $P$ es el número de núcleos de CPU).
* Cada núcleo ejecuta una instancia independiente de la función de búsqueda en su sub-lista. Como las imágenes son archivos independientes, no hay necesidad de comunicación compleja ni bloqueos entre procesos.



**(c) ¿Cuál sería la complejidad si usaras  procesadores en paralelo?**

La complejidad teórica se reduciría a:

$\mathcal{O}\left(\frac{N}{P}\right)$

El tiempo total se divide aproximadamente por el número de procesadores disponibles, asumiendo que el cuello de botella de I/O no sature el bus de datos del sistema.

---

### Pregunta 4: Seguridad de la Esteganografía

**(a) Si el atacante NO usara un encabezado predecible como "FLAG", ¿sería posible automatizar la búsqueda?**

Sería **extremadamente difícil**, y en muchos casos imposible, automatizar la búsqueda de manera determinista.

* Sin un encabezado conocido (Magic Bytes), no hay un patrón fijo de parada para el algoritmo.
* Se tendría que recurrir a análisis estadísticos (como medir la entropía del texto extraído) para intentar distinguir si los bits extraídos forman un lenguaje coherente o código binario válido, en lugar de ruido. Sin embargo, si el mensaje es corto o está comprimido, esto genera muchos falsos positivos.



**(b) ¿Cómo distinguirías entre "ruido aleatorio" y "mensaje cifrado"?**

Si el mensaje está cifrado con un algoritmo robusto (como AES), es **estadísticamente indistinguible del ruido aleatorio**.

* Tanto el ruido de una imagen generada aleatoriamente como un texto cifrado tendrán una entropía cercana al máximo (8 bits por byte) y una distribución uniforme de bits 0 y 1.
* Sin la clave de descifrado, no existe una prueba matemática eficiente para diferenciar el payload cifrado de los bits aleatorios del contenedor.



**(c) ¿Qué técnicas adicionales podría usar el atacante para dificultar la detección?**

1. **Cifrado:** Cifrar el mensaje antes de ocultarlo para eliminar patrones de texto legible.
2. **Distribución Aleatoria (Scattering):** No usar los píxeles en orden secuencial (1, 2, 3...), sino usar un generador de números pseudoaleatorios (basado en una contraseña) para elegir qué píxeles modificar.
3. **Esteganografía en otros dominios:** Usar coeficientes DCT (como en JPEG) en lugar de LSB espacial para resistir la compresión.
4. **Uso de canales específicos:** Ocultar datos solo en el canal azul (menos sensible al ojo humano) o en áreas de alta frecuencia de la imagen (bordes) donde el ruido es menos visible.
